In [ ]:
# Install required libraries
%pip install transformers>=4.41 accelerate datasets sentence-transformers


In [ ]:
# Import necessary modules
from datasets import load_dataset


In [ ]:
# Load the rotten_tomatoes dataset
dataset = load_dataset("rotten_tomatoes")


In [ ]:
# Display the DatasetDict structure
print(dataset)
print(f"\nTrain: {len(dataset['train'])} rows")
print(f"Validation: {len(dataset['validation'])} rows")
print(f"Test: {len(dataset['test'])} rows")


## Using a task-specific model


In [ ]:
from transformers import pipeline
from tqdm import tqdm
from sklearn.metrics import classification_report


In [ ]:
classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")


In [ ]:
test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]

predictions = []
for text in tqdm(test_texts, desc="Running inference"):
    result = classifier(text)[0]
    score = result["score"]
    label = result["label"].lower()
    
    if "positive" in label:
        pred = 1
    else:
        pred = 0
    
    predictions.append(pred)


In [ ]:
print(classification_report(test_labels, predictions, target_names=["negative", "positive"]))


## Classification tasks with embeddings


### A) Supervised classification with embeddings


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np


In [ ]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")


In [ ]:
train_texts = dataset["train"]["text"]
train_labels = dataset["train"]["label"]
test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]

train_embeddings = model.encode(train_texts, show_progress_bar=True)
test_embeddings = model.encode(test_texts, show_progress_bar=True)


In [ ]:
classifier = LogisticRegression(max_iter=1000)
classifier.fit(train_embeddings, train_labels)
predictions = classifier.predict(test_embeddings)


In [ ]:
print(classification_report(test_labels, predictions, target_names=["negative", "positive"]))


### B) Unsupervised use case


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

train_embeddings_neg = train_embeddings[train_labels == 0]
train_embeddings_pos = train_embeddings[train_labels == 1]

class_centroid_neg = train_embeddings_neg.mean(axis=0)
class_centroid_pos = train_embeddings_pos.mean(axis=0)

class_centroid_neg = np.nan_to_num(class_centroid_neg, nan=0.0)
class_centroid_pos = np.nan_to_num(class_centroid_pos, nan=0.0)


In [ ]:
centroids = np.vstack([class_centroid_neg, class_centroid_pos])
test_embeddings_clean = np.nan_to_num(test_embeddings, nan=0.0)
similarities = cosine_similarity(test_embeddings_clean, centroids)
predictions = similarities.argmax(axis=1)


In [ ]:
print(classification_report(test_labels, predictions, target_names=["negative", "positive"]))


### C) Zero-shot classification


In [ ]:
label_descriptions = ["A negative review", "A positive review"]
label_embeddings = model.encode(label_descriptions)


In [ ]:
similarities = cosine_similarity(test_embeddings, label_embeddings)
predictions = similarities.argmax(axis=1)


In [ ]:
print(classification_report(test_labels, predictions, target_names=["negative", "positive"]))


## Text classification with generative models


In [ ]:
%pip install groq python-dotenv


**To set your Groq API key:**

**Option 1 (Recommended):** Use Colab Secrets
- Click the 🔑 key icon in the left sidebar
- Add a new secret: `GROQ_API_KEY` = `your-api-key`

**Option 2:** Uncomment and set directly in the cell below


In [ ]:
# Uncomment and set your Groq API key here:
# os.environ["GROQ_API_KEY"] = "your-groq-api-key-here"


In [ ]:
import os

# Try to get API key from Colab secrets first
try:
    from google.colab import userdata
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
except:
    pass

# Check if API key is set
if not os.environ.get("GROQ_API_KEY"):
    raise ValueError("GROQ_API_KEY not found. Please set it using Colab secrets (recommended) or uncomment the line in the cell above.")


In [ ]:
from groq import Groq
from tqdm import tqdm
from sklearn.metrics import classification_report

if not os.environ.get("GROQ_API_KEY") or os.environ["GROQ_API_KEY"] == "":
    raise ValueError("GROQ_API_KEY is not set. Please configure it in the previous cell.")

client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [ ]:
def groq_predict(text):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a sentiment classifier. Respond with 'positive' or 'negative' only."},
            {"role": "user", "content": f"Classify the sentiment of this movie review: {text}"}
        ],
        temperature=0.1,
        max_tokens=10
    )
    prediction = response.choices[0].message.content.strip().lower()
    if "positive" in prediction:
        return "positive"
    else:
        return "negative"


In [ ]:
test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]

predictions = []
for text in tqdm(test_texts, desc="Running Groq predictions"):
    pred = groq_predict(text)
    predictions.append(pred)


In [ ]:
predictions_mapped = [1 if p == "positive" else 0 for p in predictions]
print(classification_report(test_labels, predictions_mapped, target_names=["negative", "positive"]))


## Text2Text transfert transformers


In [ ]:
import torch
from transformers import pipeline
from tqdm import tqdm
from sklearn.metrics import classification_report

device = "cuda:0" if torch.cuda.is_available() else "cpu"
t5_pipeline = pipeline("text2text-generation", model="google/flan-t5-small", device=device)


In [ ]:
def add_t5_prompt(example):
    prompt = "Is the following sentence positive or negative? "
    example["t5"] = prompt + example["text"]
    return example

dataset = dataset.map(add_t5_prompt)


In [ ]:
test_t5 = dataset["test"]["t5"]
test_labels = dataset["test"]["label"]

predictions = []
for text in tqdm(test_t5, desc="Running T5 inference"):
    result = t5_pipeline(text, max_length=10)[0]["generated_text"]
    predictions.append(result.strip().lower())


In [ ]:
predictions_mapped = []
for pred in predictions:
    if "positive" in pred:
        predictions_mapped.append(1)
    else:
        predictions_mapped.append(0)

print(classification_report(test_labels, predictions_mapped, target_names=["negative", "positive"]))


## Conclusion

This lab explored multiple approaches to text classification, demonstrating the trade-offs between different methods. Task-specific models like RoBERTa fine-tuned for sentiment analysis typically achieve the highest performance by leveraging domain-specific training. Supervised classification with embeddings offers a flexible alternative that combines pre-trained representations with task-specific classifiers. Unsupervised centroid-based methods provide a simple baseline without requiring labeled data, while zero-shot classification with label descriptions enables classification without training examples. Generative LLMs via API calls offer flexibility and ease of use but may be slower and more expensive. Text2Text transformers like T5 demonstrate how sequence-to-sequence models can be adapted for classification tasks through prompting. The choice of method depends on factors such as available training data, computational resources, latency requirements, and desired accuracy.
